 **This is the lab assignment for ENPM 690 Spring 2022. Please do not distribute this collab file without permission.**

Created on Fri Mar 28 16:58:59 2022

@author: Ruiqi


Import required packages

If you prefer to use your own complier, those are the things you probably need to do:

- You could download and install pytorch from https://pytorch.org/

- Other packages could be installed via pip or conda.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random
import os
import time

You could download the LFW dataset using the following command:

    wget http://vis-www.cs.umass.edu/lfw/lfw.tgz
    tar -xvzf lfw.tgz
    find lfw -iname "*jpg" > images_list.txt

You could also directly download and process the data using torchvision.
Please check:
  1. https://pytorch.org/vision/main/generated/torchvision.datasets.LFWPeople.html
  2. https://pytorch.org/vision/main/generated/torchvision.datasets.LFWPairs.html
  
If you directly use pytorch to download and process the data, then you would
not need to implement the following processes.


If you use the command lines to download the datasets, you will have a
dataset folder named lfw and a txt file with lines format as:

  lfw/Allan_Wagner/

  lfw/Allan_Wagner/Allen_Wagner_0001.jpg

  lfw/Alejandro_Fernandez/

  lfw/Alejandro_Fernandez/Alejandro_Fernandez_0001.jpg

  ...


In [ ]:
! wget http://vis-www.cs.umass.edu/lfw/lfw.tgz
! tar -xvzf lfw.tgz
! find lfw -iname "*jpg" > images_list.txt

**Pre-pocess data**

400 persons are in the test set and 400 persons are included in the
validation set. The remainings are used for training.

In [ ]:
images_dict = {}
count = 0
for line in open("images_list.txt","r"):
    line = line.strip()
    person = line.split("/")[-2]
    if person not in images_dict:
        images_dict[person] = [line]
    else:
        images_dict[person].append(line)
    count += 1

print("Number of unique persons = ", str(len(images_dict)))
print("NUmber of total images = ", str(count))
unique_ids = list(images_dict.keys())
val_ids = unique_ids[-800:-400]
test_ids = unique_ids[-400:]
train_ids = unique_ids[:-800]

Defining pytorch dataset class with return paired data (2 faces) and a label
(0 or 1) which verify if the faces belong to the same person or not.

In [ ]:
class LFW_dataset(torch.utils.data.Dataset):

  def __init__(self, split="train", images_dict=images_dict, ids=train_ids):
    self.split = split
    self.images_dict = images_dict
    self.ids = ids

  def __getitem__(self, index):
    id1 = self.ids[index]
    if len(self.images_dict[id1]) == 1:
      id2 = np.random.randint(0, len(self.ids))
      id2 = self.ids[id2]
      label = 0
    else:
      id2 = id1
      label = 1
    img1 = Image.open(self.images_dict[id1][0])
    img2 = Image.open(random.sample(self.images_dict[id2], 1)[0])
    img1 = transforms.ToTensor()(img1)
    img2 = transforms.ToTensor()(img2)
    if label == 0:
      label = torch.Tensor([1, 0])
    else:
      label = torch.Tensor([0, 1])
    img = torch.cat((img1, img2), 0)
    return img, label

  def __len__(self):
    return len(self.ids)

Define pytorch dataloaders

In [ ]:
train_dataset = LFW_dataset()
train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=32)
val_dataset = LFW_dataset(split="val", )
val_dataloader = torch.utils.data.DataLoader(val_dataset, shuffle=True, batch_size=32)
test_dataset = LFW_dataset(split="test", images_dict=images_dict, ids=test_ids)
test_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=32)

# for i, data in enumerate(train_dataloader, 0):
#         # Get inputs
#         inputs, lables = data
#         print(inputs.size())

# ** You need to fill out the following code to build up your own model and define your training, testing and main functions. Please do not directly use the models that stored in pytorch. You have to build your own model. Specifically, you need to define a network class with your network architecture.**

**If you are new to pytroch, you could find tutorials on https://pytorch.org/tutorials/**

Define your network architecture here (5 points): 

(Hint: you need to decide what type of network you would like to use first. If it is a CNN, then you need to define convolutional layers and fully-connected layers in the init function, and activation layers in the forward function. Please remember, your network only need to have two output: 0 or 1.)

You could use a small network model first, to see if your code works or not. Otherwise, you may waste a lot of time on training.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(6, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 32, 5)
        self.fc1 = nn.Linear(32 * 59 * 59, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


Define your training process here (5 points):

(Hint: In this part, you need to define your loss function and code for your backpropagation process. Remember to return your training loss, cause you need to print out your result after each epoch to see if they are converging)

In [ ]:
def train(model, device, train_loader, optimizer):

    loss_function = nn.MSELoss() 
    running_loss = 0
    for i, data in enumerate(train_loader, 0):
        # Get inputs
        inputs, lables = data
        inputs, lables = inputs.to(device), lables.to(device)

        # Zero the model gradient
        optimizer.zero_grad()

        # Forward + Backward + Optimize
        outputs = model(inputs)
        loss = loss_function(outputs, lables)
        loss.backward()
        optimizer.step() # Perfrom learning step

        running_loss += loss.item()
        
    running_loss = running_loss / len(train_loader)

    return running_loss

Define your testing process here (5 points):

(Hint: You need to evaluate your model at each epoch. Basically, you need to feed your test data into the model you trained before and calculate the accuracy of your model for each epoch. In this part, you still need to return the testing loss to monitor whether the results are converging or if overfitting happens).

In [ ]:
def test(model, device, test_loader):
    correct = 0
    total = 0
    loss_function = nn.MSELoss() 
    test_loss = 0 
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            images, lables = data
            images, lables = images.to(device), lables.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += lables.size(0)
            correct += (predicted.argmax() == lables.argmax(1)).type(torch.float).sum().item()

            loss = loss_function(outputs, lables)

            test_loss += loss.item()

    accuracy = (correct / total) * 100
    test_loss = test_loss / len(test_loader)

    return test_loss, accuracy

Define your main function to train your model here (5 points):

(Hint:Here you need to define your optimization method and call the train and test function for each epoch)

***Please print out your training loss, testing loss and accuracy after each epoch***


In [ ]:
start_time = time.time()
batch_size = 32
num_epoch = 30
    
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
    
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])

model = Net().to(device)
# print(model)

# You can start your code here
#################################################
lr = 0.000001
optimizer = torch.optim.Adam(model.parameters(), lr) # lr is learning rate

# Training loop

for epoch in range(num_epoch):
    running_loss = train(model, device, train_dataloader, optimizer)
    print(f'Epoch: {epoch} loss: {running_loss}')
    test_loss, model_accuracy = test(model, device, test_dataloader,)
    print(f'Epoch: {epoch} loss: {test_loss} Model Accuracy: {model_accuracy}')








#################################################

end_time = time.time()
elasped_time = end_time-start_time


In [ ]:
torch.save(model.state_dict(), f'models/model_lr{lr}.pth')

In [ ]:
model_learning_rate = 0.001
torch.load(f'models/model_lr{model_learning_rate}.pth')